# Exercise 17.5
we cannot perform PCA on the data set if it is evenly distributed on a circumference as the data is symmetrical through the origin, meaning we PC1 is a constant regardless of the position of the PC line

# Eigenfashion


In [1]:
import cv2
import numpy as np

import glob
filenames = glob.glob("/Users/nhatpham/Downloads/Man's Clothing/*.JPEG")
for i in filenames:
    img = cv2.imread(i)
    res = cv2.resize(img, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)
filenames = glob.glob("/Users/nhatpham/Downloads/Women's Clothing/*.JPEG")
for i in filenames:
    img = cv2.imread(i)
    res = cv2.resize(img, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)

In [2]:
from PIL import Image
import PIL.ImageOps

from collections import defaultdict
from glob import glob
from random import shuffle, seed
import numpy as np
import pylab as pl
import pandas as pd
import re
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.linear_model import LogisticRegression


# this is the size of all the Target.com images
STANDARD_SIZE = (138,138)
HALF_SIZE = (STANDARD_SIZE[0]/2,STANDARD_SIZE[1]/2)

def img_to_array(filename):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    img = img.resize(STANDARD_SIZE)
    img = list(img.getdata())
    img = [list(i) for i in img]
    img = np.array(img)
    s = img.shape[0] * img.shape[1]
    img_wide = img.reshape(1, s)
    return img_wide[0]

# my files are set up like "images/girls/gapkids/image1.jpg" and "images/boys/oldnavy/image1.jpg"
men_files = glob("/Users/nhatpham/Downloads/Man's Clothing/*")
women_files = glob("/Users/nhatpham/Downloads/Woman's Clothing/*")

process_file = img_to_array

raw_data = [(process_file(filename),'men',filename) for filename in men_files] + \
           [(process_file(filename),'women',filename) for filename in women_files]

# randomly order the data
seed(0)
shuffle(raw_data)

# pull out the features and the labels
data = np.array([cd for (cd,_y,f) in raw_data])
labels = np.array([_y for (cd,_y,f) in raw_data])

# find the principal components
N_COMPONENTS = 10
pca = RandomizedPCA(n_components=N_COMPONENTS, random_state=0)
X = pca.fit_transform(data)
y = [1 if label == 'men' else 0 for label in labels]



In [3]:
raw_data

[(array([204, 165, 122, ..., 168, 135,  90]),
  'women',
  "/Users/nhatpham/Downloads/Woman's Clothing/n04596852_2576.JPEG"),
 (array([19, 19, 17, ..., 28, 28, 26]),
  'women',
  "/Users/nhatpham/Downloads/Woman's Clothing/n04596852_1990.JPEG"),
 (array([139, 139, 137, ...,  91,  74,  44]),
  'women',
  "/Users/nhatpham/Downloads/Woman's Clothing/n04596852_2122.JPEG"),
 (array([195, 181, 155, ..., 189, 108,  29]),
  'women',
  "/Users/nhatpham/Downloads/Woman's Clothing/n04596852_6448.JPEG"),
 (array([ 31,  34,  27, ..., 176, 176, 138]),
  'men',
  "/Users/nhatpham/Downloads/Man's Clothing/n03746330_579.JPEG"),
 (array([255, 255, 255, ..., 255, 255, 255]),
  'men',
  "/Users/nhatpham/Downloads/Man's Clothing/n03746330_1543.JPEG"),
 (array([254, 254, 254, ..., 202, 198, 187]),
  'men',
  "/Users/nhatpham/Downloads/Man's Clothing/n03746330_5241.JPEG"),
 (array([ 15,  57, 143, ...,  17,  59, 133]),
  'men',
  "/Users/nhatpham/Downloads/Man's Clothing/n03746330_3072.JPEG"),
 (array([142, 1

In [6]:
train_split = int(len(data) * 4.0 / 5.0)

X_train = X[:train_split]
X_test = X[train_split:]
y_train = y[:train_split]
y_test = y[train_split:]

# if you wanted to use a different model, you'd specify that here
clf = LogisticRegression(penalty='l2')
clf.fit(X_train,y_train)

print ("score",clf.score(X_test,y_test))
    


score 0.6361829025844931


In [7]:
train_split = int(len(data) * 4.0 / 5.0)
X_wo_pca_train = data[:train_split]
X_wo_pca_test = data[train_split:]
clf_woPCA = LogisticRegression()
clf_woPCA.fit(X_wo_pca_train,y_train)
print ("score",clf_woPCA.score(X_wo_pca_test,y_test))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


score 0.6182902584493042


In [180]:
# and now some qualitative results

# first, let's find the model score for every shirt in our dataset
probs = list(zip(clf.decision_function(X),list(raw_data)))


girliest_girl_shirt = sorted(probs,key = lambda x: ((0 if list(list(x)[1])[1] == 'girl' else 1), list(x)[0]))[0]
girliest_boy_shirt = sorted(probs,key = lambda x: ((0 if list(list(x)[1])[1] == 'boy' else 1), list(x)[0]))[0]
boyiest_girl_shirt = sorted(probs,key = lambda x: ((0 if list(list(x)[1])[1] == 'girl' else 1), -list(x)[0]))[0]
boyiest_boy_shirt = sorted(probs,key = lambda x: ((0 if list(list(x)[1])[1] == 'girl' else 1), list(x)[0]))[0]
most_androgynous_shirt = sorted(probs,key = lambda x: abs(list(x)[0]))[0]

# and let's look at the most and least extreme clothes
cd = list(zip(X,raw_data))


least_extreme_shirt = sorted(cd,key=lambda x: sum([abs(c) for c in list(x)[0]]))[0]
most_extreme_shirt =  sorted(cd,key=lambda x: sum([abs(c) for c in list(x)[0]]),reverse=True)[0]

least_interesting_shirt = sorted(cd,key=lambda x: max([abs(c) for c in list(x)[0]]))[0]
most_interesting_shirt =  sorted(cd,key=lambda x: min([abs(c) for c in list(x)[0]]),reverse=True)[0]

# and now let's look at precision-recall
probs = list(zip(clf.decision_function(X_test),raw_data[train_split:]))
num_men = len([c for c in y_test if c == 1])
num_woman = len([c for c in y_test if c == 0])
lowest_score = round(min([list(p)[0] for p in probs]),1) - 0.1
highest_score = round(max([list(p)[0] for p in probs]),1) + 0.1
INTERVAL = 0.1

# first do the woman
score = lowest_score
while score <= highest_score:
    true_positives  = len([p for p in probs if p[0] <= score and p[1][1] == 'girl'])
    false_positives = len([p for p in probs if p[0] <= score and p[1][1] == 'boy'])
    positives = true_positives + false_positives
    if positives > 0:
        precision = 1.0 * true_positives / positives
        recall = 1.0 * true_positives / num_woman
        print ("woman",score,precision,recall)
    score += INTERVAL

# then do the men
score = highest_score
while score >= lowest_score:
    true_positives  = len([p for p in probs if p[0] >= score and p[1][1] == 'boy'])
    false_positives = len([p for p in probs if p[0] >= score and p[1][1] == 'girl'])
    positives = true_positives + false_positives
    if positives > 0:
        precision = 1.0 * true_positives / positives
        recall = 1.0 * true_positives / num_men
        print ("men",score,precision,recall)
    score -= INTERVAL

# now do both
score = lowest_score
while score <= highest_score:
    woman  = len([p for p in probs if p[0] <= score and p[1][1] == 'girl'])
    men = len([p for p in probs if p[0] <= score and p[1][1] == 'boy'])
    print (score, woman, men)
    score += INTERVAL

woman -2.1 1.0 0.003968253968253968
woman -2.0 1.0 0.007936507936507936
woman -1.9 1.0 0.007936507936507936
woman -1.7999999999999998 1.0 0.011904761904761904
woman -1.6999999999999997 0.5 0.015873015873015872
woman -1.5999999999999996 0.5 0.01984126984126984
woman -1.4999999999999996 0.5 0.01984126984126984
woman -1.3999999999999995 0.5714285714285714 0.031746031746031744
woman -1.2999999999999994 0.5882352941176471 0.03968253968253968
woman -1.1999999999999993 0.68 0.06746031746031746
woman -1.0999999999999992 0.75 0.13095238095238096
woman -0.9999999999999992 0.7833333333333333 0.1865079365079365
woman -0.8999999999999992 0.7432432432432432 0.21825396825396826
woman -0.7999999999999993 0.75 0.27380952380952384
woman -0.6999999999999993 0.7570093457943925 0.32142857142857145
woman -0.5999999999999993 0.7345132743362832 0.32936507936507936
woman -0.49999999999999933 0.7307692307692307 0.376984126984127
woman -0.39999999999999936 0.7006369426751592 0.4365079365079365
woman -0.299999999